In [4]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Milvus
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain_experimental.text_splitter import SemanticChunker

from pymilvus import connections

from milvus import default_server

default_server.start()
connections.connect("default", host="0.0.0.0")


data = ["Le banane sono buone", "Le banane sono molto buone", "Le banane sono buonissime"]

documents = []
for document in data:
    documents.append(Document(page_content="test document sentence 1", metadata={"source": "test"}))


embedder = HuggingFaceEmbeddings(model_name="BAAI/bge-m3",
                                     model_kwargs={"device": "cuda"})

vector_db = Milvus.from_documents(
        documents,
        embedder, 
        connection_args={"host":  "127.0.0.1", "port": "19530"},
        collection_name="test"
    )

query = "Come sono le banane?"
docs = vector_db.similarity_search(query)

KeyboardInterrupt: 

In [ ]:
for doc in docs:
    print(doc)


page_content='test document sentence 1' metadata={'source': 'test', 'pk': 451208543413994819}
page_content='test document sentence 1' metadata={'source': 'test', 'pk': 451208543413994820}
page_content='test document sentence 1' metadata={'source': 'test', 'pk': 451209027367206961}
page_content='test document sentence 1' metadata={'source': 'test', 'pk': 451208543413994818}


## Loading the Dataset into Milvus

In [ ]:
import os
import pandas as pd
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from pymilvus import connections

from milvus import default_server

DEFAULT_SAVE_DIR = os.getcwd() + "/work/documents/"
CODICE_PENALE_DIR = DEFAULT_SAVE_DIR + "Codice_Penale.csv"
REFERENCES_CSV_FILE = DEFAULT_SAVE_DIR + "quiz_references.csv"

default_server.start()
connections.connect("default", host="0.0.0.0")

df = pd.read_csv(CODICE_PENALE_DIR)
data = (df.iloc[:, 0] + " " + df.iloc[:, 1]).tolist()

# Process the list to create Document objects
documents = []
for document in data:
    documents.append(Document(page_content=document, metadata={"source": "csv"}))

embedder = HuggingFaceEmbeddings(model_name="BAAI/bge-m3",
                                     model_kwargs={"device": "cuda"})

# Add the documents to the Milvus database
vector_db = Milvus.from_documents(
        documents,
        embedder, 
        connection_args={"host":  "127.0.0.1", "port": "19530"},
        collection_name="csv_data"
    )

# Load the references
df_queries = pd.read_csv(REFERENCES_CSV_FILE)

queries = df_queries.iloc[:, 1].tolist()
df_results = pd.DataFrame(columns=['Question', 'Answer'])

# For each query in the list, query the database and store the question and answer in the DataFrame
for query in queries:
    docs = vector_db.similarity_search(query)
    answer = docs[0].page_content if docs else 'No match found'
    df_results = df_results.append({'Question': query, 'Answer': answer}, ignore_index=True)

